In [1]:
import warnings

import numpy as np

from astropy.coordinates import SkyCoord
from astropy.cosmology import Planck18 as cosmo
from astropy.cosmology import z_at_value
from astropy.io import fits
from astropy.table import Table, vstack
import astropy.units as u
from astropy.utils.metadata import MergeConflictWarning
from astroquery.xmatch import XMatch
import healpy as hp
from ligo.skymap.io import read_sky_map
from ligo.skymap.postprocess import find_greedy_credible_levels
import pandas as pd

import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
from lsst.geom import degrees, SpherePoint

/home/s/sanand08/.local/lib/python3.12/site-packages/ligo/skymap/io/events/ligolw.py:26: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
repo = "/repo/embargo"
collection = "u/elhoward/TOO/S251112cm/Rubin-DECam/processed"
butler = Butler(repo, collections=collection)

In [3]:
data_refs = butler.query_datasets("dia_object",where="skymap='lsst_cells_v1'")

In [4]:
len(data_refs)

527

In [5]:
"""
good = ~dia_src["slot_Shape_flag"] & \
    (dia_src["base_PsfFlux_instFlux"] / dia_src["base_PsfFlux_instFluxErr"] > snr_threshold) & \
    ~dia_src["base_PixelFlags_flag_edge"] & \
    ((dia_src_table["scienceFlux"] / dia_src_table["scienceFluxErr"]) < max_science_snr) & \
    ~dia_src_table["pixelFlags_streak"]
"""

def good_src_unfiltered(cat):
    """Good for the dia_source_unfiltered"""
    good = ~cat["slot_Shape_flag"] & \
        (cat["base_PsfFlux_instFlux"] / cat["base_PsfFlux_instFluxErr"] > snr_threshold) & \
        ~cat["base_PixelFlags_flag_edge"]

    good_cat = cat[good].copy()

    return good_cat

def good_src(cat):
    """Basic filtering criteria. Good for the dia_source tables"""
    good = ~cat["isDipole"] & ~cat["isNegative"]\
        (cat["reliability"] > 0.1)
    good_cat = cat[good].copy()
    
    return good_cat

def good_obj(cat):
    """Require at least two detections in dia_object tables"""
    good = cat["nDiaSources"] > 1

    good_cat = cat[good].copy()

    return good_cat


In [6]:
# Display columns in the DIA source catalog
dia_src = butler.get("dia_source", dataId=data_refs[10].dataId)
dia_src.columns

Index(['visit', 'detector', 'band', 'diaObjectId', 'ssObjectId',
       'parentDiaSourceId', 'midpointMjdTai', 'bboxSize',
       'timeProcessedMjdTai', 'ra', 'dec', 'raErr', 'decErr', 'ra_dec_Cov',
       'x', 'y', 'xErr', 'yErr', 'apFlux', 'apFluxErr', 'snr', 'psfFlux',
       'psfFluxErr', 'psfChi2', 'psfNdata', 'trailFlux', 'trailFluxErr',
       'trailRa', 'trailDec', 'trailLength', 'trailAngle', 'dipoleMeanFlux',
       'dipoleMeanFluxErr', 'dipoleFluxDiff', 'dipoleFluxDiffErr',
       'dipoleLength', 'dipoleAngle', 'dipoleChi2', 'isDipole', 'isNegative',
       'dipoleFitAttempted', 'dipoleNdata', 'scienceFlux', 'scienceFluxErr',
       'templateFlux', 'templateFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF',
       'iyyPSF', 'ixyPSF', 'extendedness', 'reliability', 'glint_trail',
       'pixelFlags', 'pixelFlags_offimage', 'pixelFlags_edge',
       'pixelFlags_interpolated', 'pixelFlags_saturated', 'pixelFlags_cr',
       'pixelFlags_bad', 'pixelFlags_suspect', 'pixelFlags_interpolated

In [7]:
# Display columns in DIA object catalog
dia_obj = butler.get(data_refs[10])
dia_obj.columns

Index(['ra', 'dec', 'nDiaSources', 'i_psfFluxMaxSlope', 'i_psfFluxErrMean',
       'i_psfFluxMean', 'i_psfFluxMeanErr', 'i_psfFluxNdata',
       'i_scienceFluxMean', 'i_scienceFluxMeanErr', 'i_psfFluxMin',
       'i_psfFluxMax', 'i_psfFluxSigma', 'g_psfFluxMaxSlope',
       'g_psfFluxErrMean', 'g_psfFluxMean', 'g_psfFluxMeanErr',
       'g_psfFluxNdata', 'g_scienceFluxMean', 'g_scienceFluxMeanErr',
       'g_psfFluxMin', 'g_psfFluxMax', 'g_psfFluxSigma'],
      dtype='object')

## Inside-outside calculation

In [6]:
def in_skymap(skymap_prob, good_dia_obj, dataset_type="dia_object", probability = 0.95):
    # if not good_dia_src:
    #     dia_src = butler.get(dataset_type, dataId=dr.dataId).asAstropy()
    #     good_dia_src = good_src(dia_src)
    ra_obj = good_dia_obj["ra"]
    dec_obj = good_dia_obj["dec"]
    npix = len(skymap_prob)
    nside = hp.npix2nside(npix)
    ipix = hp.ang2pix(nside, ra_obj, dec_obj, lonlat=True)
    credible_levels = find_greedy_credible_levels(skymap_prob)
    return credible_levels[ipix] <= probability

In [7]:
skymap = read_sky_map("../data/S251112cm_bilby.fits")
prob = skymap[0]

In [9]:
dia_objs_inside = []
for i, dr in enumerate(data_refs):
    dia_obj = butler.get(dr)
    if len(dia_obj) < 1: 
        continue
    good_dia_obj = good_obj(dia_obj) #--> for chained collection
    inside = in_skymap(prob, good_dia_obj=good_dia_obj)
    dia_obj_inside = good_dia_obj[inside]
    if len(dia_obj_inside) < 1:
        # print("no dia objects in data ref")
        continue
    elif len(dia_objs_inside) < 1:
       dia_objs_inside = pd.DataFrame(dia_obj_inside) 
    else:
        # print(len(dia_obj_inside))
        dia_objs_inside = pd.concat([dia_objs_inside, dia_obj_inside], ignore_index=True)

no dia objects in data ref
81
2
1
97
145
101
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
36
105
251
108
60
no dia objects in data ref
no dia objects in data ref
1
32
1
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
27
130
225
390
205
115
7
85
195
120
136
123
91
17
no dia objects in data ref
30
58
362
177
506
392
76
13
34
41
285
205
138
242
131
61
16
89
63
42
25
12
211
477
396
642
358
567
51
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
11
32
53
41
53
156
220
161
278
7
56
155
344
459
192
670
221
5
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
no dia objects in data ref
6
13
48
125
264
402
590
233
294
304
782
393
482
23
35
2
no dia objects in data ref
no dia objects in data ref
no dia objec

In [10]:
dia_objs_inside

,ra,dec,nDiaSources,g_psfFluxMaxSlope,g_psfFluxErrMean,g_psfFluxMean,g_psfFluxMeanErr,g_psfFluxNdata,g_scienceFluxMean,g_scienceFluxMeanErr,...,i_psfFluxMaxSlope,i_psfFluxErrMean,i_psfFluxMean,i_psfFluxMeanErr,i_psfFluxNdata,i_scienceFluxMean,i_scienceFluxMeanErr,i_psfFluxMin,i_psfFluxMax,i_psfFluxSigma
0,36.465145,-60.604419,2,NaN,183.125885,-340.164734,183.125885,1.0,9447.605469,192.471497,...,NaN,330.397980,1.341640e+04,330.398010,1.0,3.663455e+04,340.452698,1.341640e+04,1.341640e+04,NaN
1,36.398297,-60.611189,2,NaN,175.705093,-482.970428,175.705093,1.0,3890.145996,184.573151,...,NaN,295.287201,3.304961e+03,295.287201,1.0,7.633921e+03,304.047150,3.304961e+03,3.304961e+03,NaN
2,36.391252,-60.618925,2,NaN,175.876007,-297.567719,175.876007,1.0,3992.940674,184.828400,...,NaN,323.481750,3.156254e+03,323.481750,1.0,2.782542e+04,333.083862,3.156254e+03,3.156254e+03,NaN
3,36.365143,-60.614202,2,NaN,177.555176,7283.842773,177.555176,1.0,3966.204590,186.479645,...,NaN,300.633606,4.382465e+03,300.633606,1.0,1.056302e+04,309.366425,4.382465e+03,4.382465e+03,NaN
4,36.368163,-60.507494,2,NaN,184.049927,3705.371582,184.049927,1.0,10709.261719,193.410187,...,NaN,303.163757,4.276637e+03,303.163757,1.0,1.096454e+04,311.643097,4.276637e+03,4.276637e+03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91548,343.341474,0.814182,2,NaN,412.925720,57968.292969,412.925720,1.0,206926.375000,427.995148,...,NaN,1646.326660,1.480686e+06,1646.326660,1.0,2.312670e+06,1734.923828,1.480686e+06,1.480686e+06,NaN
91549,343.357938,0.792603,2,NaN,186.548279,-3929.481445,186.548279,1.0,-245.004074,193.455231,...,NaN,571.991394,-9.162837e+03,571.991394,1.0,-3.739226e+02,603.719421,-9.162837e+03,-9.162837e+03,NaN
91550,343.155102,0.887600,2,NaN,258.416321,232.147827,258.416321,1.0,42075.609375,264.922424,...,NaN,766.056702,-4.753509e+03,766.056702,1.0,2.105274e+05,803.460144,-4.753509e+03,-4.753509e+03,NaN
91551,343.071414,0.839093,2,NaN,201.322174,-2034.364258,201.322174,1.0,13580.139648,206.879410,...,NaN,626.081482,-9.485284e+03,626.081482,1.0,5.094725e+04,643.365234,-9.485284e+03,-9.485284e+03,NaN


In [44]:
# data_refs_inside = [dr for dr, dsm in zip(data_refs, dia_srcs_inside) if len(dsm) > 0]
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore', category=MergeConflictWarning)
#     dia_obj_inside = vstack(dia_objs_inside)

In [12]:
dia_objs_inside.to_csv("../data/dia_obj_inside95_S251112cm.csv")

In [8]:
# if the csv is already created, load it
dia_objs_inside = pd.read_csv("../data/dia_obj_inside95_S251112cm.csv")

## NED-GWF galaxy crossmatch

In [12]:
# Load NED-GWF table
url = "https://ned.ipac.caltech.edu/uri/NED::GWFglist/fits/S251112cm/4"

ned_gwf = Table(fits.getdata(url))

In [13]:
ned_gwf

objname,ra,dec,objtype,z,z_unc,z_tech,z_qual,z_qual_flag,z_refcode,ziDist,ziDist_unc,ziDist_method,ziDist_indicator,ziDist_refcode,DistMpc,DistMpc_unc,DistMpc_method,ebv,A_FUV_MWext,A_NUV_MWext,A_J_MWext,A_H_MWext,A_Ks_MWext,A_W1_MWext,A_W2_MWext,A_W3_MWext,A_W4_MWext,m_FUV,m_FUV_unc,m_NUV,m_NUV_unc,Lum_FUV,Lum_FUV_unc,Lum_NUV,Lum_NUV_unc,GALEXphot,m_J,m_J_unc,m_H,m_H_unc,m_Ks,m_Ks_unc,Lum_J,Lum_J_unc,Lum_H,Lum_H_unc,Lum_Ks,Lum_Ks_unc,tMASSphot,m_W1,m_W1_unc,m_W2,m_W2_unc,m_W3,m_W3_unc,m_W4,m_W4_unc,Lum_W1,Lum_W1_unc,Lum_W2,Lum_W2_unc,Lum_W3,Lum_W3_unc,Lum_W4,Lum_W4_unc,WISEphot,SFR_W4,SFR_W4_unc,SFR_hybrid,SFR_hybrid_unc,ET_flag,Mstar,Mstar_unc,MLratio,P_2D,P_3D,P_2D_csum,P_3D_csum,P_Mstar,P_SFR,P_sSFR,P_LumW1,P_3D_Mstar,P_3D_SFR,P_3D_sSFR,P_3D_LumW1
str30,float64,float64,str6,float32,float32,str4,bool,bool,str19,float64,float64,str30,str1,str19,float32,float32,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,bool,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
AGC 208747,159.2775,20.5161111111111,G,0.00032689283,nan,UNKN,False,False,2016A&A...596A.117A,nan,nan,,,,1.4084027,nan,Redshift,0.019383822,0.1552131,0.17419715,0.015746957,0.009809612,0.006728239,0.003847457,0.0026839837,0.0010237546,0.0005326131,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,False,nan,nan,0.5,1.105000506557702,1.9688844829228506e-06,0.7095791258224945,0.5917971263975689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WISEA J013915.89-471751.8,24.8162476,-47.2977329,G,0.00054,0.000213,SPEC,False,False,1997AJ....114.1313C,nan,nan,,,,2.3269491,0.91746837,Redshift,0.012746077,0.102062345,0.11454555,0.010354611,0.0064504347,0.0044242386,0.002529944,0.0017648875,0.0006731828,0.00035022647,18.5441,0.0597547,18.2229,0.0352552,1.9534029466684373e+39,1.0751128615171526e+38,1.7762852484337926e+39,5.768010621731761e+37,ASC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,13.812,0.023,13.785,0.073,11.963,0.369,9.063,99.0,5.368503599881293e+38,1.1372900202386011e+37,2.2232108368809907e+38,1.4948363570663178e+37,8.731533184456437e+37,2.9676112951414574e+37,1.7435972343054267e+38,nan,APER,3.4063504186743166e-05,nan,0.00011631947236994556,nan,False,1698893.5442662318,35990.19051387978,0.5,2.294141496059659,1.0706827784233411e-06,0.5523232371090585,0.7362317337576566,9.644972019085916e-09,8.457623026546877e-09,5.7096645434090956e-05,9.524965514323448e-09,1.032670543921029e-14,9.055431320920437e-15,6.113239497202487e-11,1.0198216541262338e-14
GALEXASC J235059.08-264835.2,357.74679166667,-26.810416666669997,G,0.0006,0.000297,SPEC,True,True,20032dF...C...0000C,nan,nan,,,,2.5856192,1.2792869,Redshift,0.018108554,0.14500159,0.16273667,0.01471096,0.009164235,0.006285586,0.0035943314,0.0025074035,0.00095640146,0.0004975723,21.5012,0.345958,21.6129,0.320332,1.6469606468366552e+38,5.248035263440326e+37,1.0100544128882755e+38,2.980129881071741e+37,ASC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,,nan,nan,nan,nan,False,nan,nan,0.5,3.19992875866416,2.0271092063679064e-06,0.44733279462273773,0.5854630746238607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UGC 07577,186.92042,43.495556,G,0.00065,1e-06,SPEC,False,False,2003A&A...401..483H,2.6443064212799072,0.031269438398829535,Wavg,P,Mix,2.6443064,0.03126944,zIndependent,0.017462263,0.1398265,0.15692863,0.014185928,0.008837165,0.0060612545,0.0034660504,0.0024179148,0.0009222676,0.000479814,15.018273,0.0008275661,14.63243,0.002857707,6.7185866035

In [14]:
def load_and_match_dia_obj_against_cat(cat, good_dia_obj, dataset_type="dia_object", radius=30 * u.arcsec, debug=False):
    # if not good_dia_src:
    #     dia_src = butler.get(dataset_type, dataId=dr.dataId).asAstropy()
    #     good_dia_src = good_src(dia_src)
    
    dia_coord = SkyCoord(np.array(good_dia_obj["ra"]) * u.deg, np.array(good_dia_obj["dec"]) * u.deg)
    cat_coord = SkyCoord(cat["ra"] * u.deg, cat["dec"] * u.deg)
    
    idx, sep2d, _ = dia_coord.match_to_catalog_sky(cat_coord)

    close_enough = sep2d < radius
    matching_idx = idx[close_enough]

    if debug:
        print(len(dia_coord))
        print(cat_coord[364])
        print(idx)
        print(sep2d)
        print(matching_idx)
        print("----")

    matched_dia_obj = good_dia_obj[close_enough].copy()
    matched_cat = cat[matching_idx].copy()

    return matched_dia_obj, matched_cat
    

In [15]:
# Use an approximate matching radius given redshift.
z = z_at_value(cosmo.luminosity_distance, 93 * u.Mpc)
matching_radius_physical = 50 * u.kpc
matching_radius_angular = matching_radius_physical * cosmo.arcsec_per_kpc_proper(z)
print(matching_radius_physical, matching_radius_angular)

50.0 kpc 115.52553420764819 arcsec


In [16]:
# So let's call that 116"
ned_matching_radius = 116 * u.arcsec

In [17]:
ned_dia_obj_match, ned_cat_match = load_and_match_dia_obj_against_cat(ned_gwf, dia_objs_inside, dataset_type="dia_object", radius=ned_matching_radius)

In [18]:
ned_dia_obj_match

,Unnamed: 0,ra,dec,nDiaSources,g_psfFluxMaxSlope,g_psfFluxErrMean,g_psfFluxMean,g_psfFluxMeanErr,g_psfFluxNdata,g_scienceFluxMean,...,i_psfFluxMaxSlope,i_psfFluxErrMean,i_psfFluxMean,i_psfFluxMeanErr,i_psfFluxNdata,i_scienceFluxMean,i_scienceFluxMeanErr,i_psfFluxMin,i_psfFluxMax,i_psfFluxSigma
274,274,34.924312,-59.108785,2,NaN,175.28600,4494.713379,175.285995,1.0,8481.864258,...,NaN,294.75717,5794.444824,294.757172,1.0,1.158063e+04,302.226440,5794.444824,5794.444824,NaN
328,328,35.096283,-58.691890,2,NaN,442.04930,33429.804688,442.049286,1.0,292936.718750,...,NaN,790.68760,34082.855469,790.687622,1.0,5.952999e+05,813.174438,34082.855469,34082.855469,NaN
381,381,35.208249,-58.042877,2,NaN,680.34900,16792.884766,680.348999,1.0,548579.875000,...,NaN,1144.26660,151092.218750,1144.266602,1.0,1.618785e+06,1172.172729,151092.218750,151092.218750,NaN
485,485,30.512960,-57.933589,2,NaN,246.06642,-3804.890381,246.066422,1.0,6904.773438,...,NaN,471.93265,-100.132797,471.932648,1.0,3.113353e+04,492.468597,-100.132790,-100.132790,NaN
486,486,30.480150,-57.920678,2,NaN,261.94608,-743.134705,261.946075,1.0,12102.372070,...,NaN,486.07785,1866.519043,486.077850,1.0,3.776304e+04,505.121765,1866.519043,1866.519043,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91325,91325,337.925552,0.219457,2,NaN,NaN,NaN,NaN,NaN,NaN,...,-89301.984382,396.29663,-6922.125000,279.990082,2.0,3.293273e+02,285.780060,-6943.685059,-6902.506836,29.117400
91465,91465,337.142791,0.687793,2,NaN,NaN,NaN,NaN,NaN,NaN,...,-695644.382552,404.62012,6219.848633,285.263092,2.0,1.303805e+04,293.729706,6045.237305,6366.007324,226.818649
91495,91495,342.425599,0.167221,2,NaN,340.96863,-1042.414307,340.968628,1.0,129942.304688,...,NaN,709.57806,30319.070312,709.578064,1.0,2.191008e+05,748.875793,30319.068359,30319.068359,NaN
91521,91521,342.666604,0.826672,2,NaN,561.59800,79706.523438,561.598022,1.0,442081.875000,...,NaN,1567.40550,880858.375000,1567.405518,1.0,2.057204e+06,1649.474609,880858.375000,880858.375000,NaN


In [19]:
ned_cat_match

objname,ra,dec,objtype,z,z_unc,z_tech,z_qual,z_qual_flag,z_refcode,ziDist,ziDist_unc,ziDist_method,ziDist_indicator,ziDist_refcode,DistMpc,DistMpc_unc,DistMpc_method,ebv,A_FUV_MWext,A_NUV_MWext,A_J_MWext,A_H_MWext,A_Ks_MWext,A_W1_MWext,A_W2_MWext,A_W3_MWext,A_W4_MWext,m_FUV,m_FUV_unc,m_NUV,m_NUV_unc,Lum_FUV,Lum_FUV_unc,Lum_NUV,Lum_NUV_unc,GALEXphot,m_J,m_J_unc,m_H,m_H_unc,m_Ks,m_Ks_unc,Lum_J,Lum_J_unc,Lum_H,Lum_H_unc,Lum_Ks,Lum_Ks_unc,tMASSphot,m_W1,m_W1_unc,m_W2,m_W2_unc,m_W3,m_W3_unc,m_W4,m_W4_unc,Lum_W1,Lum_W1_unc,Lum_W2,Lum_W2_unc,Lum_W3,Lum_W3_unc,Lum_W4,Lum_W4_unc,WISEphot,SFR_W4,SFR_W4_unc,SFR_hybrid,SFR_hybrid_unc,ET_flag,Mstar,Mstar_unc,MLratio,P_2D,P_3D,P_2D_csum,P_3D_csum,P_Mstar,P_SFR,P_sSFR,P_LumW1,P_3D_Mstar,P_3D_SFR,P_3D_sSFR,P_3D_LumW1
str30,float64,float64,str6,float32,float32,str4,bool,bool,str19,float64,float64,str30,str1,str19,float32,float32,str30,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,str5,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str5,float64,float64,float64,float64,bool,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MFGC 01807,34.9323948,-59.1341075,G,0.027189,0.00015,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,119.55667,0.6461045,Redshift,0.023697319,0.18975279,0.21296138,0.019251138,0.011992552,0.008225478,0.0047036344,0.0032812525,0.0012515715,0.0006511358,nan,nan,nan,nan,nan,nan,nan,nan,,13.794,0.113,12.97,0.126,12.773,0.162,2.0455109780201283e+43,2.128974361803611e+42,2.0717900822386294e+43,2.4043986330542284e+42,1.240832491625399e+43,1.8514770327532625e+42,XSC,12.655,0.01,12.599,0.022,9.705,0.035,8.587,0.281,4.121889749593139e+42,3.7965272798076734e+40,1.7521069170064587e+42,3.5503684702813025e+40,1.8454212192754974e+42,5.949133515454913e+40,7.137403570940461e+41,1.8472969610974877e+41,APER,0.1394387256630723,0.03608941705712727,nan,nan,False,13043954903.775757,120143268.34834409,0.5,0.634488320757125,5.093678238047204e-07,0.8377745063295571,0.865036644975257,7.405324511929534e-05,3.462122306897953e-05,0.00010709264214468265,7.313184575229882e-05,3.772034031209299e-11,1.7634937052103885e-11,5.454954607473469e-11,3.7251009121670935e-11
ESO 115- G 005,35.0651383,-58.7082084,G,0.029961,3.3e-05,SPEC,False,False,1996ApJS..107...97M,115.59737396240234,2.4944382578492945,Wavg,S,Mix,115.597374,2.4944382,zIndependent,0.022238858,0.1780744,0.19985461,0.018066322,0.011254467,0.007719239,0.0044141477,0.0030793068,0.001174543,0.00061106146,17.2458,0.0501429,16.8564,0.0249922,1.7093798974834753e+43,7.894746971329975e+41,1.6693922042303765e+43,3.842846163201934e+41,ASC,12.839,0.066,12.269,0.08,11.71,0.091,4.60337230836771e+43,2.798402667015735e+42,3.6914644663863146e+43,2.7200621913734002e+42,3.086446835790855e+43,2.586963452053016e+42,XSC,11.606,0.006,11.457,0.011,7.705,0.01,5.774,0.026,1.0123467778599416e+43,5.594621500553624e+40,4.688439122866009e+42,4.750191551826369e+40,1.0884613658887167e+43,1.0025433581754557e+41,8.90168939405111e+42,2.1317481479908583e+41,APER,1.7390640910493183,0.04164655147155884,2.2860845857220555,0.05060708737682863,False,32036290438.605743,177044984.19473493,0.5,0.634488320757125,5.952687076287839e-07,0.8377745063295571,0.8437304668811442,0.0001818766843387573,0.0004317919971023922,2.108921354676956e-05,0.00017961370367462577,1.0826549883414035e-10,2.570322640795926e-10,1.2553748892892958e-11,1.0691841725881384e-10
AM 0219-581,35.2085367,-58.0430766,G,0.021105,0.00015,SPEC,False,False,20096dF...C...0000J,nan,nan,,,,92.38256,0.6461045,Redshift,0.03020056,0.24182655,0.27140427,0.024534218,0.015283662,0.010482792,0.00599445,0.004181725,0.0015950395,0.0008298267,20.9338,0.340217,20.2045,0.145299,3.876358591226053e+41,1.2147

In [58]:
# ned_dia_src_matches = []
# ned_cat_matches = []

# for dr in data_refs_inside:
#     if len(dia_obj) < 1: 
#         continue
#     matched_dia_src, matched_cat = load_and_match_dia_obj_against_cat(ned_gwf, dr=dr, dataset_type="dia_source_unstandardized", radius=ned_matching_radius)
#     ned_dia_src_matches.append(matched_dia_src)
#     ned_cat_matches.append(matched_cat)

In [59]:
# ned_dr_match = [dr for dr, dsm in zip(data_refs_inside, ned_dia_src_matches) if len(dsm) > 0]

In [32]:
ned_dia_obj_match.to_csv("../data/dia_src_match_116arcsec_S251112cm.csv")

In [9]:
ned_dia_obj_match = pd.read_csv("../data/dia_src_match_116arcsec_S251112cm.csv")

## Gaia crossmatch

In [26]:
# check what Gaia DR3 catalogs are available in astroquery.XMatch using the CDS identifier
catalogs_available = XMatch.get_available_tables()
for name in catalogs_available:
    if "I/355" in name:
        print(name)

I/355/paramp
II/355/ugriz-2
I/355/xpsummary
I/355/paramsup
I/355/gaiadr3


In [10]:
def load_and_match_dia_obj_against_gaia(good_dia_obj, dataset_type="dia_object", radius= 3.0):
    # if not good_dia_src:
    #     dia_src = butler.get("dia_source_unstandardized", dataId=dr.dataId).asAstropy()
    #     good_dia_src = good_src(dia_src)
        
    input_table = Table.from_pandas(good_dia_obj)
    gaia_table = XMatch.query(cat1=input_table, cat2="vizier:I/355/gaiadr3", max_distance=radius * u.arcsec, colRA1='ra', colDec1='dec')
    
    return gaia_table
    

In [11]:
# table of all gaia dr3 sources that crossmatch
gaia_match = load_and_match_dia_obj_against_gaia(dia_objs_inside, radius = 3.0)

In [12]:
gaia_match

angDist,Unnamed: 0,ra,dec,nDiaSources,g_psfFluxMaxSlope,g_psfFluxErrMean,g_psfFluxMean,g_psfFluxMeanErr,g_psfFluxNdata,g_scienceFluxMean,g_scienceFluxMeanErr,g_psfFluxMin,g_psfFluxMax,g_psfFluxSigma,i_psfFluxMaxSlope,i_psfFluxErrMean,i_psfFluxMean,i_psfFluxMeanErr,i_psfFluxNdata,i_scienceFluxMean,i_scienceFluxMeanErr,i_psfFluxMin,i_psfFluxMax,i_psfFluxSigma,DR3Name,RAdeg,DEdeg,errHalfMaj,errHalfMin,errPosAng,SolID,Source,RandomI,e_RAdeg,e_DEdeg,Plx,e_Plx,RPlx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,nueff,pscol,e_pscol,RApscolCorr,DEpscolCorr,PlxpscolCorr,pmRApscolCorr,pmDEpscolCorr,MatchObsA,Nper,amax,MatchObs,IPDgofha,IPDgofhp,IPDfmp,IPDfow,RUWE,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E(BP/RP),NBPcont,NBPblend,NRPcont,NRPblend,Mode,BP-RP,BP-G,G-RP,RV,e_RV,n_RV,o_RV,o_RVd,RVNper,RVS/N,RVgof,RVchi2,RVTdur,RVamp,RVtempTeff,RVtemplogg,RVtemp[Fe/H],Vatmparam,vbroad,e_Vbroad,o_Vbroad,GRVSmag,e_GRVSmag,o_GRVSmag,RVSS/N,VarFlag,PQSO,PGal,PSS,Teff,b_Teff,B_Teff,logg,b_logg,B_logg,[Fe/H],b_[Fe/H],B_[Fe/H],Dist,b_Dist,B_Dist,A0,b_A0,B_A0,AG,b_AG,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,arcsec,arcsec,deg,,,,mas,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,deg,,,,,,electron/s,electron/s,,mag,,,electron/s,electron/s,,mag,,,electron/s,electron/s,,mag,,,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,dex(cm / s2),dex,,km / s,km / s,,mag,mag,,,,,,,K,K,K,dex(cm / s2),dex(cm / s2),dex(cm / s2),dex,dex,dex,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,,deg,deg,,,
float64,int32,float64,float64,int16,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,str28,float64,float64,float32,float32,float32,int64,int64,int32,float32,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,uint8,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,uint8,float32,int16,float32,float32,uint8,uint8,float32,uint8,int16,float64,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,float32,int16,int16,int16,int16,uint8,float32,float32,float32,float32,float32,uint8,int16,int16,uint8,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,uint8,float32,float32,int16,float32,str13,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str7,float64,float64,float64,float64,float32
0.361494,16,34.34732662262453,-60.29624670898554,2,--,592.1314,-1926.67919921875,592.1314086914062,1.0,251995.203125,623.34423828125,-1926.67919921875,-1926.67919921875,--,--,872.64685,65928.96875,872.6468505859375,1.0,442540.0625,891.7189331054688,65928.96875,65928.96875,--,Gaia DR3 4713819417309470080,34.34713702327,-60.29628215869,0.0,0.0,90.0,1636148068921376768,4713819417309470080,1392309914,0.0604,0.0534,0.6101,0.061,9.9962,11.274,11.252,0.08,-0.706,0.065,-0.0015,0.0134,-0.3279,-0.1605,0.155,-0.1571,0.016,0.1241,-0.0526,-0.0363,354,0,353,1,-0.8095,340.04,0.0,0.0,31,True,1.488,--,--,--,--,--,--,--,41,25,0.113958,59,0.02193,75.17,0,0,0.969,0,494,2188.50347,1.55708,1405.52,17.336998,0.002862,52,1009.06,7.80173,129.338,17.828754,0.008846,53,1654.98,8.07636,204.916,16.700914,0.006508,1.217,0,1,0,1,0,1.12784,0.491756,0.636084,--,--,0,0,0,0,--,--,--,--,--,--,--,--,0,--,--,0,--,--,0,--,NOT_AVAILABLE,0.0,0.0,0.999983,4738.7,4732.

In [30]:
gaia_match.columns

<TableColumns names=('angDist','Unnamed: 0','ra','dec','nDiaSources','g_psfFluxMaxSlope','g_psfFluxErrMean','g_psfFluxMean','g_psfFluxMeanErr','g_psfFluxNdata','g_scienceFluxMean','g_scienceFluxMeanErr','g_psfFluxMin','g_psfFluxMax','g_psfFluxSigma','i_psfFluxMaxSlope','i_psfFluxErrMean','i_psfFluxMean','i_psfFluxMeanErr','i_psfFluxNdata','i_scienceFluxMean','i_scienceFluxMeanErr','i_psfFluxMin','i_psfFluxMax','i_psfFluxSigma','DR3Name','RAdeg','DEdeg','errHalfMaj','errHalfMin','errPosAng','SolID','Source','RandomI','e_RAdeg','e_DEdeg','Plx','e_Plx','RPlx','PM','pmRA','e_pmRA','pmDE','e_pmDE','RADEcor','RAPlxcor','RApmRAcor','RApmDEcor','DEPlxcor','DEpmRAcor','DEpmDEcor','PlxpmRAcor','PlxpmDEcor','pmRApmDEcor','NAL','NAC','NgAL','NbAL','gofAL','chi2AL','epsi','sepsi','Solved','APF','nueff','pscol','e_pscol','RApscolCorr','DEpscolCorr','PlxpscolCorr','pmRApscolCorr','pmDEpscolCorr','MatchObsA','Nper','amax','MatchObs','IPDgofha','IPDgofhp','IPDfmp','IPDfow','RUWE','Dup','o_Gmag','FG','e

In [13]:
# filter out sources likely to be a single star (can change the cuts here)
mask = (gaia_match['Plx'] is not None) & (gaia_match['e_Plx'] < 1.0)
gaia_stars = gaia_match[mask]

In [14]:
gaia_stars

angDist,Unnamed: 0,ra,dec,nDiaSources,g_psfFluxMaxSlope,g_psfFluxErrMean,g_psfFluxMean,g_psfFluxMeanErr,g_psfFluxNdata,g_scienceFluxMean,g_scienceFluxMeanErr,g_psfFluxMin,g_psfFluxMax,g_psfFluxSigma,i_psfFluxMaxSlope,i_psfFluxErrMean,i_psfFluxMean,i_psfFluxMeanErr,i_psfFluxNdata,i_scienceFluxMean,i_scienceFluxMeanErr,i_psfFluxMin,i_psfFluxMax,i_psfFluxSigma,DR3Name,RAdeg,DEdeg,errHalfMaj,errHalfMin,errPosAng,SolID,Source,RandomI,e_RAdeg,e_DEdeg,Plx,e_Plx,RPlx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,nueff,pscol,e_pscol,RApscolCorr,DEpscolCorr,PlxpscolCorr,pmRApscolCorr,pmDEpscolCorr,MatchObsA,Nper,amax,MatchObs,IPDgofha,IPDgofhp,IPDfmp,IPDfow,RUWE,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E(BP/RP),NBPcont,NBPblend,NRPcont,NRPblend,Mode,BP-RP,BP-G,G-RP,RV,e_RV,n_RV,o_RV,o_RVd,RVNper,RVS/N,RVgof,RVchi2,RVTdur,RVamp,RVtempTeff,RVtemplogg,RVtemp[Fe/H],Vatmparam,vbroad,e_Vbroad,o_Vbroad,GRVSmag,e_GRVSmag,o_GRVSmag,RVSS/N,VarFlag,PQSO,PGal,PSS,Teff,b_Teff,B_Teff,logg,b_logg,B_logg,[Fe/H],b_[Fe/H],B_[Fe/H],Dist,b_Dist,B_Dist,A0,b_A0,B_A0,AG,b_AG,B_AG,E(BP-RP),b_E(BP-RP),B_E(BP-RP),Lib,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,deg,deg,arcsec,arcsec,deg,,,,mas,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,deg,,,,,,electron/s,electron/s,,mag,,,electron/s,electron/s,,mag,,,electron/s,electron/s,,mag,,,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,dex(cm / s2),dex,,km / s,km / s,,mag,mag,,,,,,,K,K,K,dex(cm / s2),dex(cm / s2),dex(cm / s2),dex,dex,dex,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,,deg,deg,,,
float64,int32,float64,float64,int16,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float64,str28,float64,float64,float32,float32,float32,int64,int64,int32,float32,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,uint8,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,uint8,float32,int16,float32,float32,uint8,uint8,float32,uint8,int16,float64,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,int16,float64,float32,float32,float32,float32,float32,int16,int16,int16,int16,uint8,float32,float32,float32,float32,float32,uint8,int16,int16,uint8,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,uint8,float32,float32,int16,float32,str13,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str7,float64,float64,float64,float64,float32
0.361494,16,34.34732662262453,-60.29624670898554,2,--,592.1314,-1926.67919921875,592.1314086914062,1.0,251995.203125,623.34423828125,-1926.67919921875,-1926.67919921875,--,--,872.64685,65928.96875,872.6468505859375,1.0,442540.0625,891.7189331054688,65928.96875,65928.96875,--,Gaia DR3 4713819417309470080,34.34713702327,-60.29628215869,0.0,0.0,90.0,1636148068921376768,4713819417309470080,1392309914,0.0604,0.0534,0.6101,0.061,9.9962,11.274,11.252,0.08,-0.706,0.065,-0.0015,0.0134,-0.3279,-0.1605,0.155,-0.1571,0.016,0.1241,-0.0526,-0.0363,354,0,353,1,-0.8095,340.04,0.0,0.0,31,True,1.488,--,--,--,--,--,--,--,41,25,0.113958,59,0.02193,75.17,0,0,0.969,0,494,2188.50347,1.55708,1405.52,17.336998,0.002862,52,1009.06,7.80173,129.338,17.828754,0.008846,53,1654.98,8.07636,204.916,16.700914,0.006508,1.217,0,1,0,1,0,1.12784,0.491756,0.636084,--,--,0,0,0,0,--,--,--,--,--,--,--,--,0,--,--,0,--,--,0,--,NOT_AVAILABLE,0.0,0.0,0.999983,4738.7,4732.

In [16]:
dia_objs_coords = SkyCoord(ra=np.array(dia_objs_inside["ra"]), dec=np.array(dia_objs_inside["dec"]), unit=(u.deg, u.deg))
gaia_coords = SkyCoord(ra=gaia_stars["ra"], dec=gaia_stars["dec"], unit=(u.deg, u.deg))
mask = [False if coord in gaia_coords else True for coord in dia_objs_coords]
dia_objs_filtered_gaia = dia_objs_inside[mask]

KeyboardInterrupt: 

In [71]:
# gaia_cat_matches = []
# dia_srcs_filtered_gaia = []
# for dr in ned_dr_match:
#     unmatched_dia_src, matched_cat = load_and_match_dia_src_against_gaia(dr=dr, dataset_type="dia_source_unstandardized")
#     dia_srcs_filtered_gaia.append(unmatched_dia_src)
#     gaia_cat_matches.append(matched_cat)

In [74]:
# gaia_dr_filtered = [dr for dr, dsm in zip(ned_dr_match, dia_srcs_filtered_gaia) if len(dsm) > 0]
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore', category=MergeConflictWarning)
#     dia_src_filtered_gaia = vstack(dia_srcs_filtered_gaia)
#     gaia_cat_match = vstack(gaia_cat_matches)

In [75]:
dia_src_filtered_gaia

id,coord_ra,coord_dec,parent,is_negative,detect_isPrimary,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xErr,base_SdssCentroid_yErr,base_SdssCentroid_flag,base_CircularApertureFlux_flag_badCentroid,base_PsfFlux_flag_badCentroid,ext_shapeHSM_HsmPsfMoments_flag_badCentroid,ext_shapeHSM_HsmSourceMoments_flag_badCentroid,base_SdssCentroid_flag_edge,base_CircularApertureFlux_flag_badCentroid_edge,base_PsfFlux_flag_badCentroid_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_edge,base_SdssCentroid_flag_noSecondDerivative,base_CircularApertureFlux_flag_badCentroid_noSecondDerivative,base_PsfFlux_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_noSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_noSecondDerivative,base_SdssCentroid_flag_almostNoSecondDerivative,base_CircularApertureFlux_flag_badCentroid_almostNoSecondDerivative,base_PsfFlux_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_almostNoSecondDerivative,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_almostNoSecondDerivative,base_SdssCentroid_flag_notAtMaximum,base_CircularApertureFlux_flag_badCentroid_notAtMaximum,base_PsfFlux_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_notAtMaximum,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_notAtMaximum,base_SdssCentroid_flag_near_edge,base_CircularApertureFlux_flag_badCentroid_near_edge,base_PsfFlux_flag_badCentroid_near_edge,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_near_edge,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_near_edge,base_SdssCentroid_flag_resetToPeak,base_CircularApertureFlux_flag_badCentroid_resetToPeak,base_PsfFlux_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_resetToPeak,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_resetToPeak,base_SdssCentroid_flag_badError,base_CircularApertureFlux_flag_badCentroid_badError,base_PsfFlux_flag_badCentroid_badError,ext_shapeHSM_HsmPsfMoments_flag_badCentroid_badError,ext_shapeHSM_HsmSourceMoments_flag_badCentroid_badError,ip_diffim_DipoleFit_pos_instFlux,slot_Centroid_pos_instFlux,ip_diffim_DipoleFit_pos_instFluxErr,slot_Centroid_pos_instFluxErr,ip_diffim_DipoleFit_neg_instFlux,slot_Centroid_neg_instFlux,ip_diffim_DipoleFit_neg_instFluxErr,slot_Centroid_neg_instFluxErr,ip_diffim_DipoleFit_instFlux,slot_Centroid_instFlux,ip_diffim_DipoleFit_instFluxErr,slot_Centroid_instFluxErr,ip_diffim_DipoleFit_pos_x,slot_Centroid_pos_x,ip_diffim_DipoleFit_pos_y,slot_Centroid_pos_y,ip_diffim_DipoleFit_pos_xErr,slot_Centroid_pos_xErr,ip_diffim_DipoleFit_pos_yErr,slot_Centroid_pos_yErr,ip_diffim_DipoleFit_neg_x,slot_Centroid_neg_x,ip_diffim_DipoleFit_neg_y,slot_Centroid_neg_y,ip_diffim_DipoleFit_neg_xErr,slot_Centroid_neg_xErr,ip_diffim_DipoleFit_neg_yErr,slot_Centroid_neg_yErr,ip_diffim_DipoleFit_x,slot_Centroid_x,ip_diffim_DipoleFit_y,slot_Centroid_y,ip_diffim_DipoleFit_xErr,slot_Centroid_xErr,ip_diffim_DipoleFit_yErr,slot_Centroid_yErr,ip_diffim_DipoleFit_orientation,slot_Centroid_orientation,ip_diffim_DipoleFit_separation,slot_Centroid_separation,ip_diffim_DipoleFit_chi2dof,slot_Centroid_chi2dof,ip_diffim_DipoleFit_nData,slot_Centroid_nData,ip_diffim_DipoleFit_signalToNoise,slot_Centroid_signalToNoise,ip_diffim_DipoleFit_classification,slot_Centroid_classification,ip_diffim_DipoleFit_classificationAttempted,slot_Centroid_classificationAttempted,ip_diffim_DipoleFit_flag,slot_Centroid_flag,ip_diffim_DipoleFit_flag_edge,slot_Centroid_flag_edge,coord_raErr,coord_decErr,coord_ra_dec_Cov,ext_shapeHSM_HsmPsfMoments_flag,slot_PsfShape_flag,ext_shapeHSM_HsmPsfMoments_flag_no_pixels,slot_PsfShape_flag_no_pixels,ext_shapeHSM_HsmPsfMoments_flag_not_contained,slot_PsfShape_flag_not_contained,ext_shapeHSM_HsmPsfMoments_flag_parent_source,slot_PsfShape_flag_parent_source,ext_shapeHSM_HsmPsfMoments_flag_galsim,slot_PsfShape_flag_galsim,ext_shapeHSM_HsmPsfMoments_flag_invalid_param,slot_PsfShape_flag_invalid_p

## WISE / Milliquas crossmatch

In [14]:
# check what Gaia DR3 catalogs are available in astroquery.XMatch using the CDS identifier
for name in catalogs_available:
    if "J/ApJS/234/23/r90cat" in name:
        print(name)

## Distance consistency (LS crossmatch / distant galaxies in the refcat)

In [ ]:
# 0.8 " is a typical size for galaxy in Rubin
# 0.145 kpc / arcsec appropriate for the distance to the galaxy
# discard hosts that < 1 " across -- unlikely to be associated??